<a href="https://colab.research.google.com/github/daniiltroilov/eda_online_store/blob/main/Atlantis_Odyssey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Теоретический блок

1. LTV (Lifetime value) - прогнозируемая не точная метрика показывающая сколько в среднем идёт дохода на взятый временной ряд пользователя

LTV = СРЕДНИЙ ЧЕК * 30 ДНЕЙ * СРЕДНЕЕ КОЛИЧЕСТВО ПРОДАЖ

2. ARPPU - метрика показывающая средний чек платящих пользователей

При изменениях снижения ARPPU смотрим на две другие метрики reveneu и payment count: если увеличивается количество продаж и растёт суммарный доход, то всё хорошо. Могло быть вызвано снижением цен и более выгодными предложениями.

3. Базовая ситуация снижения CPI и retention первого дня: означает некачественный и незаинтересованный трафик.

Смотрим LTV если он больше CPI, то мы всё ещё зарабатываем.
Но обычно некачественный трафик приводит к быстрому оттоку, уменьшению количества средних чеков, уменьшению средних чеков на пользователя.

# Практический блок

Импорт библиотек и файлов, подключение к базе данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

! gdown --id 1-0bx3o3zT6ZD1AoT0QaXFc4b1wJ5sVda
con = sqlite3.connect('/content/test.db')

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-0bx3o3zT6ZD1AoT0QaXFc4b1wJ5sVda
From (redirected): https://drive.google.com/uc?id=1-0bx3o3zT6ZD1AoT0QaXFc4b1wJ5sVda&confirm=t&uuid=2ea73884-ce32-4d56-b2c1-3d140250a90d
To: /content/test.db
100% 114M/114M [00:00<00:00, 185MB/s]


Упрощающая функция

In [ ]:
def select(sql):
  return pd.read_sql_query(sql, con)
sql = '''SELECT name FROM sqlite_master WHERE type='table' '''
select(sql)

,name
0,level_up
1,payment
2,quest_complete
3,quest_start
4,session_close
5,install


Быстро пробежался посмотрел пропущенные значения

In [ ]:
select('''select * from install''').isnull().sum()

,0
user_id,0
reg_time,0


Расчёт всех метрик (сообщения в print: массив NumPy переведён в float)

In [ ]:
duration = '''select duration from session_close'''
duration_avg_min = float(select(duration).mean().values / 60)

after_7_day = '''
with retention as (
select install.user_id, date(open_time) as open_time_date,
dense_rank() over (partition by install.user_id order by date(open_time)) as rank
from install
join session_close on install.user_id = session_close.user_id
)

select retention.user_id, min(open_time_date) as open_time_date from retention
where rank > 6
group by retention.user_id
'''
user_reg = '''SELECT distinct user_id FROM install'''

retention = len(select(after_7_day)) / len(select(user_reg)) * 100

sessions_avg = float(select('''
with user_sessions_count as (
select user_id, count(close_time) as sessions_cnt from session_close group by user_id)
select avg(sessions_cnt) from user_sessions_count
''').values)

quests_avg = float(select('''
with user_quests_count as (
select user_id, count(quest) as quests_cnt from quest_complete group by user_id)
select avg(quests_cnt) from user_quests_count
''').values)

total = '''select sum(amount) as total from payment'''
users_all = '''select distinct count(user_id) from install'''
ARPU = float((select(total).values / select(users_all).values))

total = '''select sum(amount) as total from payment'''
users_all_pay = '''select distinct count(user_id) from payment'''
ARPPU = float((select(total).values / select(users_all_pay).values))

Conversion_rate = float((select(users_all_pay).values / select(users_all).values) * 100)

transaction_all = '''select count(amount) from payment '''
APV = float((select(total).values)/select(transaction_all).values)

time_to_pay = float(select('''with time_to_pay as(
select payment.user_id, round(JULIANDAY(time) - JULIANDAY(reg_time), 2) * 24 * 60 as diff from payment
join install on install.user_id = payment.user_id)
select avg(diff) from time_to_pay''').values)

print(f'''
\n\n
      Вовлеченность
-Retention: {retention:.2f} %
-Duration_avg: {duration_avg_min:.2f} минут
-Sessions_avg: {sessions_avg:.1f} сессий
-Quests_avg: {quests_avg:.1f} на пользователя

      Монетизация:
-ARPU: {ARPU:.4f} (допустим $)
-ARPPU: {ARPPU:.2f} (допустим $)
-Conversion_rate: {Conversion_rate:.2f} %
-APV: {APV:.2f} (допусти $)
-Time_To_Pay: {time_to_pay:.1f} минут

по APV и ARPPU сразу можно сделать вывод, что каждый платящий пользователь совершил по одной покупке
\n\n\n\n''')

import matplotlib.pyplot as plt
import seaborn as sns

<ipython-input-93-f47cb9a9ee5a>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  duration_avg_min = float(select(duration).mean().values / 60)






      Вовлеченность
-Retention: 19.42 %
-Duration_avg: 12.51 минут
-Sessions_avg: 11.5 сессий
-Quests_avg: 15.4 на пользователя

      Монетизация:
-ARPU: 0.0123 (допустим $)
-ARPPU: 0.55 (допустим $)
-Conversion_rate: 2.25 %
-APV: 0.55 (допусти $)
-Time_To_Pay: 4125.2 минут

по APV и ARPPU сразу можно сделать вывод, что каждый платящий пользователь совершил по одной покупке







<ipython-input-93-f47cb9a9ee5a>:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sessions_avg = float(select('''
<ipython-input-93-f47cb9a9ee5a>:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  quests_avg = float(select('''
<ipython-input-93-f47cb9a9ee5a>:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ARPU = float((select(total).values / select(users_all).values))
<ipython-input-93-f47cb9a9ee5a>:38: DeprecationWarning: Conversion of an array with ndim > 0 to a

Расчёт проблемных точек:

In [ ]:
df_compl_q = select(''' select * from quest_complete''')
df_start_q = select(''' select * from quest_start''')
df_start_q['time'] = pd.to_datetime(df_start_q['time'], format='ISO8601')
df_compl_q['time'] = pd.to_datetime(df_compl_q['time'], format='ISO8601')
df_merged = pd.merge(df_start_q, df_compl_q, on=['user_id', 'quest'], suffixes=('_start', '_compl'), how='left')
df_isnull = df_merged.loc[df_merged['time_compl'].isnull()]
df_isnull['time_compl'] = df_isnull['time_compl'].fillna(1)
top10_churn_quest = df_isnull.groupby('quest')['time_compl'].agg('sum').sort_values(ascending=False).head(10)
top10_churn_quest

<ipython-input-94-4dcea18a1afa>:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_isnull['time_compl'] = df_isnull['time_compl'].fillna(1)
<ipython-input-94-4dcea18a1afa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isnull['time_compl'] = df_isnull['time_compl'].fillna(1)


,time_compl
quest,
quest_3,3950
quest_1,1964
quest_0,1349
quest_6,1085
quest_26,1015
quest_4,1013
quest_17,828
quest_13,818
quest_30,669


Тут я считал только среднее только тех кто тратил времени на выполнение выше среднего по всем игрокам

In [ ]:
df_merged['time_diff_sec'] = (df_merged['time_compl'] - df_merged['time_start']).dt.total_seconds()
df_hard_quest = df_merged.loc[df_merged['time_diff_sec'] > df_merged['time_diff_sec'].mean()]
top10_quest_time = (df_hard_quest.groupby('quest', as_index=False)['time_diff_sec'].agg('mean').sort_values(by='time_diff_sec', ascending=False).head(10))
top10_quest_time

,quest,time_diff_sec
12,quest_2,140181.192460
0,quest_0,132767.605721
23,quest_3,118894.551052
1,quest_1,117520.977049
44,quest_5,104921.849849
54,quest_6,104611.583522
34,quest_4,97204.742430
19,quest_26,96760.251729
24,quest_30,90976.021829
2,quest_10,90611.877628


In [ ]:
difficult_quest = df_merged.loc[df_merged['time_compl'].isna()]['quest']
top10_quest_leave = difficult_quest.value_counts().head(10)
difficult_quest_leave_time = pd.merge(left = top10_quest_leave, right=top10_quest_time, how='inner', on='quest')
difficult_quest_leave_time['time_diff_h'] = difficult_quest_leave_time['time_diff_sec'] /(60*60)
difficult_quest_leave_time

,quest,count,time_diff_sec,time_diff_h
0,quest_3,3950,118894.551052,33.026264
1,quest_1,1964,117520.977049,32.644716
2,quest_0,1349,132767.605721,36.879890
3,quest_6,1085,104611.583522,29.058773
4,quest_26,1015,96760.251729,26.877848
5,quest_4,1013,97204.742430,27.001317
6,quest_30,669,90976.021829,25.271117


# Результат и Гипотеза

Совпали 6 квестов по двум метрикам: покинутые и время потраченное.

Требуется установить причины трудности игроков к завершению квеста.

Особенно на начальном этапе главная задача удержать игроков.


Как целевые метрики в A/B тесте я выделил: уменьшение churn_rate и avg_time по конкретным уровням. Увелечение вовлечённости.

Я бы более уделил внимание вовлечённости и монетезации:

DAU, просмотр рекламы и количество заходов в магазин (Пользователи думают над платными предложениями)

Дальнейший анализ я бы построил на построении визуализаций, прогнозировании LTV с большей информацией, проведению А/Б теста, командному планированию и решению

# 6 новых отслеживаемых событий

1. Время попытки не завершённого платежа
2. Время открытия магазина
3. Время повышения уровня
4. Время сообщения в чате
5. Время начала просмотра рекламного видео
6. Количество приведенных в игру друзей

# SQL Блок

In [ ]:
select('''SELECT name FROM sqlite_master WHERE type='table' ''')

,name
0,level_up
1,payment
2,quest_complete
3,quest_start
4,session_close
5,install


1. Необходимо рассчитать ретеншн по первым 7 дням. За активность игрока в
определенный день считать старт сессии.
Результат: день - процент активных игроков.

In [ ]:
select('''with users_in as
(select install.user_id, date(open_time) as open_time_date,
dense_rank() over (partition by install.user_id order by date(open_time)) as day
from install
join session_close on install.user_id = session_close.user_id
),

all_reg as
(select count(distinct user_id) as user_cnt from install
)

select day,count(distinct user_id) * 1.0/ user_cnt * 100 as retention_percent
from users_in, all_reg
where day between 1 and 7
group by day, user_cnt
order by day
''')

,day,retention_percent
0,1,99.861803
1,2,50.846858
2,3,36.352242
3,4,29.885907
4,5,25.646794
5,6,22.333280
6,7,19.415073


2. Необходимо рассчитать среднее кол-во сессий на активного игрока по дням с
момента регистрации.

In [ ]:
select('''
with user_daily_sessions as
(select install.user_id, date(sc.open_time) as session_date, count(*) AS sessions_count
from install
join session_close sc on install.user_id = sc.user_id
group by install.user_id, date(sc.open_time)
),
days_since_install as
(select user_id, session_date, sessions_count,
dense_rank() over (partition by user_id order by session_date) as day
from user_daily_sessions
)

select day, round(avg(sessions_count), 2) as avg_sessions
from days_since_install
group by day
order by day
''')

,day,avg_sessions
0,1,2.05
1,2,3.57
2,3,3.82
3,4,4.03
4,5,4.13
5,6,4.25
6,7,4.47
7,8,4.71
8,9,5.07
9,10,5.12


3. Необходимо рассчитать среднюю длительность сессии в зависимости от
порядкового номера сессии.
Результат: номер сессии - средняя продолжительность.

In [ ]:
select('''with numbered_sessions as
(select user_id, open_time, close_time, duration,
row_number() over (partition by user_id order by open_time) as session_id
from session_close
)

select session_id, avg(duration) as avg_duration
from numbered_sessions
group by session_id
order by session_id''')

,session_id,avg_duration
0,1,1258.442148
1,2,959.872747
2,3,835.098814
3,4,772.186315
4,5,761.101442
...,...,...
244,245,119.973000
245,246,30.873000
246,247,47.552000
247,248,95.802000


4. Необходимо рассчитать среднее кол-во сессий перед первым платежом.
Результат: среднее кол-во сессий перед первым платежом.

In [ ]:
select('''with first_payments as
(select user_id, min(time) as first_payment_time
from payment
group by user_id
),
sessions_before_payment as
(select sc.user_id, count(*) as sessions_count
from session_close sc
join first_payments fp on sc.user_id = fp.user_id
where sc.open_time < fp.first_payment_time
group by sc.user_id
)

select avg(sessions_count) as avg_sessions
from sessions_before_payment
''')

,avg_sessions
0,7.931193


5. Необходимо рассчитать кол-во сконвертированных игроков в плательщики по
уровням.
Результат: level - кол-во игроков, кто совершил первый платеж на данном
уровне.

In [ ]:
select('''with first_payments as
(select user_id, min(time) as first_payment_time
from payment
group by user_id
),
user_level_at_payment as
(select fp.user_id, lu.level
from first_payments fp
join level_up lu on fp.user_id = lu.user_id
where lu.time <= fp.first_payment_time
group by fp.user_id
having lu.time = max(lu.time)
)

select level, count(user_id) as converted_users_count
from user_level_at_payment
group by level
order by level
''')

,level,converted_users_count
0,2.0,2
1,3.0,8
2,4.0,41
3,5.0,58
4,6.0,14
5,7.0,39
6,8.0,83
7,9.0,47
8,10.0,14
9,11.0,4


6. *Необходимо рассчитать суммарное ревенью игроков в зависимости от квестов,
которые были активны в момент совершения покупки. При наличии нескольких
активных квестов разделить ревенью в равной степени на каждый квест.
Результат: quest - суммарное кол-во ревенью.

In [ ]:
select('''select user_id, time, quest, 'start' as event_type from quest_start
union all
select user_id, time, quest, 'complete' as event_type from quest_complete ''')

,user_id,time,quest,event_type
0,531655022,2020-10-12 03:31:30.152001,quest_0,start
1,531655022,2020-10-12 03:32:12.735001,quest_1,start
2,531655022,2020-10-12 03:33:24.633000,quest_2,start
3,531655022,2020-10-12 03:33:32.126001,quest_3,start
4,531655022,2020-10-12 03:37:53.645002,quest_4,start
...,...,...,...,...
826027,743071603,2020-10-09 16:26:10.575002,quest_14,complete
826028,743071603,2020-10-09 16:33:28.583001,quest_15,complete
826029,743071603,2020-10-09 16:39:07.563002,quest_16,complete
826030,743071603,2020-10-09 17:39:16.240002,quest_17,complete


In [ ]:
select('''with quest_events as
(select user_id, time, quest, 'start' as event_type from quest_start
union all
select user_id, time, quest, 'complete' as event_type from quest_complete
),
quest_time as
(select user_id, quest,
max(case when event_type = 'start' then time end) as start_time,
min(case when event_type = 'complete' then time end) as complete_time
from quest_events
group by user_id, quest
),
active_quests_for_payments as
(select p.user_id, p.time as payment_time, p.amount, qt.quest,
count(*) over (partition by p.user_id, p.time) as active_quests_count
from payment p
join quest_time qt on p.user_id = qt.user_id
where qt.start_time <= p.time
and ((qt.complete_time is null) or (qt.complete_time > p.time))
)

select quest, round(sum(amount * 1.0 / active_quests_count), 3) as revenue
from active_quests_for_payments
group by quest
order by revenue
''')

,quest,revenue
0,quest_1,0.305
1,quest_4,0.305
2,quest_48,0.305
3,quest_53,0.305
4,quest_56,0.305
5,quest_71,0.305
6,quest_72,0.305
7,quest_73,0.305
8,quest_76,0.305
9,quest_40,0.609
